In [13]:
import sys
sys.path.append('../')
#import primes
from matplotlib import pyplot as plt
from lab3.sha1 import sha1
from lab2.client.crypto import rsa
import pickle as pkl
from shutil import copyfile
import numpy as np

p = 47

Для заданного M (табл. 1) определить значения a и b,
которые позволяют построить эллиптическую группу EM(a, b).

Для найденных в задании 3 параметров
сгенерировать все элементы эллиптической группы EM(a, b)

In [14]:
def test_prime(n):
    t = int(np.sqrt(n)) + 1
    for i in range(2,t):
        if n % i == 0:
            return False
    return True

def gen_a_b():
    pairs = []
    for i in range(p):
        for j in range(p):
            if ((4 * (i ** 3) + 27 * (j ** 2))%p != 0): #4a^3 + 27*b^2 != 0
                pairs.append((i,j))
    return pairs

In [15]:
ab = gen_a_b()
a, b = ab[np.random.randint(0,len(ab))]
print('A: ',a," B: ",b)

t = 2 * np.sqrt(p)
roots = {}
for i in range(1,p):
    t = (i * i) % p
    if t not in roots:
        roots[t] = [i]
    else:
        roots[t].append(i)
points = []
for i in range(p):
    t = (i ** 3 + a * i + b)%p
    if t in roots:
        for s in roots[t]:
            points.append((i,s))
print((points))

A:  11  B:  19
[(2, 7), (2, 40), (3, 19), (3, 28), (7, 4), (7, 43), (8, 14), (8, 33), (9, 1), (9, 46), (10, 1), (10, 46), (11, 22), (11, 25), (13, 3), (13, 44), (14, 12), (14, 35), (15, 9), (15, 38), (16, 22), (16, 25), (19, 15), (19, 32), (20, 22), (20, 25), (21, 8), (21, 39), (24, 17), (24, 30), (26, 16), (26, 31), (27, 20), (27, 27), (28, 1), (28, 46), (30, 18), (30, 29), (31, 20), (31, 27), (32, 2), (32, 45), (36, 20), (36, 27), (37, 15), (37, 32), (38, 15), (38, 32), (42, 11), (42, 36), (44, 10), (44, 37), (45, 6), (45, 41), (46, 17), (46, 30)]
A:  11  B:  19
[(2, 7), (2, 40), (3, 19), (3, 28), (7, 4), (7, 43), (8, 14), (8, 33), (9, 1), (9, 46), (10, 1), (10, 46), (11, 22), (11, 25), (13, 3), (13, 44), (14, 12), (14, 35), (15, 9), (15, 38), (16, 22), (16, 25), (19, 15), (19, 32), (20, 22), (20, 25), (21, 8), (21, 39), (24, 17), (24, 30), (26, 16), (26, 31), (27, 20), (27, 27), (28, 1), (28, 46), (30, 18), (30, 29), (31, 20), (31, 27), (32, 2), (32, 45), (36, 20), (36, 27), (37, 15

In [16]:
def extended_euclidean_algorithm(a, b):
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = b, a

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    return old_r, old_s, old_t


def inverse_of(n, p):
    gcd, x, y = extended_euclidean_algorithm(n, p)
    assert (n * x + p * y) % p == gcd

    if gcd != 1:
        # Или n равно 0, или p не является простым.
        return None
    else:
        return x % p
    
def is_on_curve(point):
    if point is None:
        return True

    x, y = point

    return (y * y - x * x * x - a * x - b) % p == 0


def point_neg(point):
    assert is_on_curve(point)

    if point is None:
        return None

    x, y = point
    result = (x, -y % p)

    assert is_on_curve(result)
    return result


def point_add(point1, point2):
    assert is_on_curve(point1)
    assert is_on_curve(point2)

    if point1 is None:
        return point2
    if point2 is None:
        return point1

    x1, y1 = point1
    x2, y2 = point2

    if x1 == x2 and y1 != y2:
        return None

    if x1 == x2:
        m = (3 * x1 * x1 + a) * inverse_of(2 * y1, p)
    else:
        # This is the case point1 != point2.
        m = (y1 - y2) * inverse_of(x1 - x2, p)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % p,-y3 % p)

    assert is_on_curve(result)

    return result


def scalar_mult(k, point):
    assert is_on_curve(point)

    if k % p == 0 or point is None:
        return None

    if k < 0:
        return scalar_mult(-k, point_neg(point))

    result = None
    addend = point

    while k:
        if k & 1:
            # Add.
            result = point_add(result, addend)

        # Double.
        addend = point_add(addend, addend)
        k >>= 1
    assert is_on_curve(result)
    return result

def to_byte_list(data):
    return list(bytearray(data, encoding='utf-8'))

def binary_array_to_str(binary_array):

    c_array = [str(elem) for elem in binary_array]
    res = ''.join(c_array)
    return res



Реализовать алгоритм обмена ключами для эллиптической группы EM(a,b).

Разработать алгоритм цифровой подписи на основе эллиптической группы EM(a, b).

1. чекаем порядок точки. выбираем наибольший 

In [17]:
G = None
maxk = 2
for pt in points:
    k = 1
    temp = pt
    while temp is not None:
        k += 1
        temp = point_add(temp,pt)
    if (k > maxk and test_prime(k)):
        G = pt
        maxk = k
print(G,maxk)

(9, 1) 19
(9, 1) 19


2. 

In [18]:
d = np.random.randint(1,maxk)
H = scalar_mult(d,G)

In [19]:
print('p ',p)
print('a ',a)
print('b ',b)
print('G ',G)
print('n ',maxk)
print('H ',H)

p  47
a  11
b  19
G  (9, 1)
n  19
H  (9, 1)
p  47
a  11
b  19
G  (9, 1)
n  19
H  (9, 1)


Цифровая подпись на эллиптических кривых

In [20]:
def sign(hsh):
    z = hsh % maxk
    r = 0
    s = 0

    while s == 0:
        while r == 0:
            k = np.random.randint(1,maxk)
            P = scalar_mult(k,G)
            r = P[0] % maxk
        s = (inverse_of(k,maxk) * (z + r * d)) % maxk
    return (r,s)

def check_sign(hsh,sgn):
    r,s = sgn
    z = hsh % maxk
    u1 = (inverse_of(s,maxk) * z) % maxk
    u2 = (inverse_of(s,maxk) * r) % maxk
    P = point_add(scalar_mult(u1,G),scalar_mult(u2,H))
    return r == P[0] % maxk

### User A

In [21]:
text = ''
for i in open("f1.txt"):
    text += i

msg = bytes(text,'utf-8')


hash_msg = to_byte_list(sha1(bytes(msg)))
hash_ = int(binary_array_to_str(hash_msg))


In [22]:
msg

b'Non-motivated functions\nThe non-motivated purposes of art are those that are integral to being human'

b'Non-motivated functions\nThe non-motivated purposes of art are those that are integral to being human'

In [29]:
msg_to_send = [msg ,sign(hash_)]
msg_to_send[0], msg_to_send[1][0]
msg_to_send = [msg_to_send[0], (msg_to_send[1][0], msg_to_send[1][1]+1)]
msg_to_send

[b'Non-motivated functions\nThe non-motivated purposes of art are those that are integral to being human',
 (7, 4)]

[b'Non-motivated functions\nThe non-motivated purposes of art are those that are integral to being human',
 (7, 4)]

### User B

In [30]:
x = to_byte_list(sha1(msg_to_send[0]))
hash_ = int(binary_array_to_str(x))
check_sign(hash_,msg_to_send[1])

False

False